# Diode ring mixer
Minicircuits : [ADE-1+, Level 7, SMT Double Balanced Mixer, RF/LO Freq 0.5 - 500 MHz](https://www.minicircuits.com/WebStore/dashboard.html?model=ADE-1%2B)

Explanation of the operation can be found in : 
* [#260: RF Diode Mixer LO Drive Level & Conversion Loss | 1dB Compression | Distortion](https://www.youtube.com/watch?v=GvadQpkZ8l0)
* [#167: How a Diode Ring Mixer works | Mixer operation theory and measurement](https://www.youtube.com/watch?v=junuEwmQVQ8)

The operation of a diode ring mixer is also explained in [Analog Devices : Activity: Diode Ring Modulator - ADALM2000](https://wiki.analog.com/university/courses/electronics/electronics_lab_diode_ring_modulator).  Mind that in this explanation, the LO-port is the port that is connected to the diode ring.  This only works if the LO-signal is symmetrical around 0V, which is not the case for the SI5351.  

Because we'll be using an SI5351 which doesn't generate a negative voltage, we will connect the SI5351 to the LO-port of the mixer, as shown in the ADE-1+ datasheet.

## Discrete implementation
Using machine wound inductors and diode arrays, a discrete implementation of a diode ring mixer can be made.  The following components are used:

### Diode selection
  * 4 diodes in a SOT-363 package : for better matching
  * avoiding schottky diodes because of their low forward voltage drop, which would result in a lower allowed signal level (BAS40, BAS70)
  * BAV99 is quite fast, with a reverse recovery time of 4ns (Schottky BAS7004SH6 is apparently only 100ns).  
  * 1N4148 has comparable speed, to the BAV99
  * Fastest common diodes are the SD101 (1ns reverse recovery time).  These are single diodes (SOD123 or SOD323 package), so 4 of them are needed.

### Ethernet transformer
* 2x ALT4532P-181-T05G ethernet transformer : work down to 100kHz (1dB loss compared to 1MHz)

I ran simulations with 1N5819 because I didn't know any better back then.  These diodes are too slow for this application.

### References
* [Homebrew mixer](https://pa3fwm.nl/technotes/tn33a-ethernet-hardware.html) : error in antenna connection
* [Jenny List's up converter](https://github.com/JennyList/LanguageSpy/blob/master/Hardware/one-inch-HF-to-VHF-converter/upconverter%20schematic.png)

## Driving the LO port
* [Reducing IMD in high-level mixers](https://www.robkalmeijer.nl/techniek/electronica/radiotechniek/hambladen/qex/2001/05_06/page45/index.html)
* LO drive should be as symmetrical as possible (50% duty cycle). A square wave is ideal.
* A 50% duty cycle square wave has no even harmonics, only odd harmonics.
* A 49% duty cycle square wave has a second harmonic at -16dBc.
* Experiment with the drive level of the SI5351 to find the best performance.
* Second harmonic can be removed by [using a very large source termination resistor (e.g. 470 ohm)](https://www.youtube.com/watch?v=epkQQfynh7A), but this will make for a nearly sinosoidal waveform.
* A simple low-pass filter to filter it out can not be used, because we need the 3rd, 5th, and 7th harmonics to drive the mixer.
* An alternative is to use a source termination resistor to provide a nice clean waveform without over/undershoot.
* The ADE-1+ mixer requires a minimum of 7dBm of LO drive, specified into a 50 ohm load.
* Will the waveform be closer to 50% duty cycle if the clock signal is doubled and a flip-flop is used to divide the signal?  A flip-flop has some delay as well.

In [1]:
import math
R = 50
Pdrive_dBm = 7
Pdrive = 10**(Pdrive_dBm/10) * 1e-3
Vdrive_rms = math.sqrt(Pdrive* R)
Vdrive_pp = Vdrive_rms * 2 # square wave
print("peak-to-peak LO drive voltage: %.2e V" % (Vdrive_pp))

peak-to-peak LO drive voltage: 1.00e+00 V



## Termination
EMRFD : S11 should be -10dB. Q should be between 1 and 10.

### Bandpass/bandstop Diplexer
Highpass/lowpass diplexers are not suitable for this application.  So J.J. Carr, RF components and circuits, fig 5.32 can not be used.

* J.J. Carr, RF components and circuits, p. 116
* J.J.Carr Secrets of RF circuit design, §20 Using the DBM (p. 347) : diplexer, pre-and post-mixer amplifiers

* 3dB pad (reduces NF by 3dB): see Mini-circuits FAQ
* EMRFD : 3dB pads applied to all three ports of the mixer.
* broadband amplifier 
* terminate all three ports
* [Reducing IMD in high-level mixers](https://www.robkalmeijer.nl/techniek/electronica/radiotechniek/hambladen/qex/2001/05_06/page45/index.html)

https://www.eevblog.com/forum/rf-microwave/double-balanced-mixers-various-questions-probems-with-my-first-attempt/
* Terminate the IF-port into a diplexer. The IF port needs a good broadband 50ohm match over the widest bandwidth you can get.
* Terminate the RF-port into a well matched 50ohm bandpass filter, or 50ohm source impedance.
Don't worry about the LO source impedance, it is far less important. Just make sure you have enough LO to drive the diodes, and a square wave with good sharp, symmetrical edges will give approx. 10 to 20dB of improvement in the intermodulation performance.
Make your transformers identical. They aren't just transformers, they're transmission lines. They are doing an impedance transformation, and they need to be as tightly matched as you can possibly make them. Trifiliar windings are a must.

#### Diplexer option 1

In [2]:
import math
# Sayre §6.4 : this only provides matching from the LO-port.  The output of the filter must be 50ohm.  There's no backward matching (S22 is 0dB)
# Hayward's IRFD, fig 4.26b proposes double ended termination to solve this.

BW = 200e3  # Bandwidth
f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
# Q = f_0 / BW # Q factor of the filter
Q = 10 # Q factor of the filter

# The formulas given here are lumped filter image parameters.  These are explained well in Sayre §6.2.3

print("Q = ", Q)
R_L = 50 # load resistance (IF port of the mixer)
L_1 = R_L / (ω * Q) # inductance at center frequency
print("L_1 = ", L_1 * 1e6, "uH")
C_1 = 1 / (ω**2 * L_1) # capacitance at center frequency
print("C_1 = ", C_1 * 1e12, "pF")
L_2 = Q * R_L / ω # inductance at center frequency
print("L_2 = ", L_2 * 1e6, "uH")
C_2 = 1 / (ω**2 * L_2) # capacitance at center frequency
print("C_2 = ", C_2 * 1e12, "pF")

Q =  10
L_1 =  0.07437146873453052 uH
C_1 =  2974.858749381222 pF
L_2 =  7.437146873453053 uH
C_2 =  29.74858749381222 pF


#### Diplexer option 2

In [3]:
# References: Bandpass / bandstop diplexer
# * EMRFD, p.3.38
# * J.J.Carr : "RF Componenents and Circuits" fig. 5.31

# S11 equals S22, which are both very good.

def calc_filter(f_0, Q):
    ω = 2 * math.pi * f_0
    R = 50 # load resistance
    L = 50 * Q / ω # inductance at center frequency
    print("L = ", L * 1e6, "uH")
    C = 1 / (ω**2 * L) # capacitance at center frequency
    print("C = ", C * 1e12, "pF")
    Cp = L / R**2
    print("Cp = ", Cp * 1e12, "pF")
    Lp = 1 / (ω**2 * Cp)
    print("Lp = ", Lp * 1e6, "uH")

calc_filter(10.7e6, 10) # IF filter port
BW = 20e6  # Bandwidth
calc_filter(98e6, 98e6 / BW) # RF filter port

L =  7.437146873453053 uH
C =  29.74858749381222 pF
Cp =  2974.858749381221 pF
Lp =  0.07437146873453054 uH
L =  0.39788735772973843 uH
C =  6.628694006326335 pF
Cp =  159.15494309189538 pF
Lp =  0.01657173501581584 uH


#### Diplexer option 3

In [4]:
# J.J.Carr : "Secrets of RF Circuit Design" p. 195, fig. 20-3
# Similar to J.J.Carr : "RF Componenents and Circuits" fig. 5.31, but with a different component values.

# The component values are wrong.

f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
L = 50 / ω # inductance at center frequency
print("L = ", L * 1e6, "uH")
C = 1 / (ω * 50) # capacitance at center frequency
print("C = ", C * 1e12, "pF")

L =  0.7437146873453053 uH
C =  297.48587493812215 pF


# Diplexer actual design

Calculations according to [Diplexer option 2](#diplexer-option-2)

## IF port diplexer

In [5]:
Q = 10 # Q factor of the filter
f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
R = 50 # load resistance

# Select L for highest Q at 10.7MHz → L = 
L = 50 * Q / ω # inductance at center frequency
print("L = ", L * 1e9, "nH")

L = 6.8e-6 # close to 6.8nH
print("User selected L = ", L * 1e9, "nH")

# Calculate C for 10.7MHz
C = 1 / (ω**2 * L) # capacitance at center frequency
print("C = ", C * 1e12, "pF")

Lp = R / (ω * Q) # inductance at center frequency
print("Lp = ", Lp * 1e9, "nH")

# Select Lp for highest Q at 10.7MHz → L = 744912168G : 68nH
Lp = 68e-9 # close to 74.4nH
print("User selected Lp = ", Lp * 1e9, "nH")

# Calculate Cp for 10.7MHz
Cp = 1 / (ω**2 * Lp) # capacitance at center frequency
print("Cp = ", Cp * 1e9, "nF")

L =  7437.146873453053 nH
User selected L =  6800.0 nH
C =  32.5359727160662 pF
Lp =  74.37146873453052 nH
User selected Lp =  68.0 nH
Cp =  3.2535972716066195 nF


### IF diplexer measurements

<figure>
    <img src="../diplexer_filter_IF_port/doc/nanovna-h4_2025-08-15_17-48-01.png" alt="IF port diplexer S21 & S11" width="400"/>
    <figcaption>Figure 1: IF port diplexer S21 & S11</figcaption>
</figure>

## RF port diplexer

In [6]:
BW = 20e6
f_0 = 98e6  # RF frequency, middle of the band
Q = f_0 / BW # Q factor of the filter
ω = 2 * math.pi * f_0
R = 50 # load resistance

# Select L for highest Q at 98MHz
L = 50 * Q / ω # inductance at center frequency
print("L = ", L * 1e9, "nH")
L = 390e-9 # L = 744762239A : 390nH
print("User selected L = ", L * 1e9, "nH")

# Calculate C for 98MHz
C = 1 / (ω**2 * L) # capacitance at center frequency
print("C = ", C * 1e12, "pF")

Lp = R / (ω * Q) # inductance at center frequency
print("Lp = ", Lp * 1e9, "nH")
# # Select Lp for highest Q at 98MHz
Lp = 17.5e-9 # L = 744914117 : 17.5nH
print("User selected Lp = ", Lp * 1e9, "nH")

# Calculate Cp for 98MHz
Cp = 1 / (ω**2 * Lp) # capacitance at center frequency
print("Cp = ", Cp * 1e12, "pF")




L =  397.8873577297384 nH
User selected L =  390.0 nH
C =  6.762752675323434 pF
Lp =  16.571735015815843 nH
User selected Lp =  17.5 nH
Cp =  150.71277390720797 pF


### RF diplexer measurements

<figure>
    <img src="../diplexer_filter_RF_port/doc/nanovna-h4_2025-08-15_17-11-01.png" alt="RF port diplexer S21 & S11" width="400"/>
    <figcaption>Figure 2: RF port diplexer S21 & S11</figcaption>
</figure>

# Measurements
## ADE-1+ mixer
The function generator is set to sine wave output, 900mVpp into 50 ohm.  It's not possible to use square wave output, because the function generator only produces a square wave up to 25MHz. 
Using higher amplitude doesn't bring the IF output up, but it does increase the distortion products. 

<figure>
  <img src="./measurement/NO_RF.bmp" alt="Local oscillator only" width="600"/>
  <figcaption>IF port output with 87.3MHz Local oscillator only</figcaption>
</figure>

The 3rd, 4th and 5th harmonic of the LO signal are visible.  All of this is with the duplexer in place.  One might wonder how bad it would be without the duplexer.

<figure>
  <img src="./measurement/2.bmp" alt="" width="600"/>
  <figcaption>IF port output with LO and RF signal</figcaption>
</figure>

The RF output signal is 3.5mVpp (function generator setting), -46.1dBm (spectrum analyzer reading) at 98MHz.  This means that we have a conversion loss of 10.7dB (-46.1dBm - (-56.8dBm)).  The desired difference frequency is at 10.7MHz, with some distortion products visible as well.

In the spectrum plot, we see the desired difference frequency at 10.7MHz, but also the sum frequency at 185.3MHz.  The harmonics of the LO signal are also visible.


# ~~Gilbert cell mixer~~
* NJM2594
* ONsemi : MC1496 / LM1496
  * requires high supply voltage (12V and or -8V)
  * LT5560 with Balun : WBC1-1TL 1:1

[Discrete-Component Mixer design](https://ham.stackexchange.com/questions/23256/discrete-component-mixer-design)
[Discrete Gilbert cell mixer with SD5000](https://www.edn.com/a-guide-to-using-fets-for-voltage-controlled-circuits-part-3/)

https://radioaficionsdr.es.tl/Mezcladores-de-RF.htm
https://www.qsl.net/va3iul/RF%20Mixers/RF_Mixers.pdf : mixer termination in last pages

# ~~Active Dual gate MOSFET mixer~~
Dual gate MOSFETs are obsolete, but they can be replaced by 2N-channel JFETs
JFETs on Digikey : https://www.digikey.be/en/products/filter/transistors/fets-mosfets/rf-fets-mosfets/285?s=N4IgjCBcoKwOwAYqgMZQGYEMA2BnApgDQgD2UA2iHGAJwBsALHCALrEAOALlCAMqcAnAJYA7AOYgAvsQC0AJmQg0kQQFcipCiBghiNVtJAyAzIuVqNZSJTBIWkh0A
* [Mixer testing of FM-receiver: BF990A](https://www.rfcandy.biz/communication/mxtest.html)
* [understanding-a-mosfet-mixer](https://hackaday.com/2018/06/11/understanding-a-mosfet-mixer/)

# ~~Passive JFET mixer~~
[pe4140ds](https://www.psemi.com/pdf/datasheets/pe4140ds.pdf)


# References
Jessop p.24 : mixer

https://www.qsl.net/va3iul/Homebrew_RF_Circuit_Design_Ideas/Homebrew_RF_Circuit_Design_Ideas.htm#Receivers

